Players Data Scraping

In [1]:
import requests
import json
import pandas as pd
import os

In [2]:
season_names = ['2014', '2015', '2016', '2017', '2018', '2019']
gws = ['3', '5', '10']
leagues = ['EPL', 'La_liga', 'Bundesliga', 'Serie_A', 'Ligue_1']